### EEVE Model Basic Testing

Code to test the baseline functionality of the EEVE model by yanolja.

In [1]:
# Import modules 

from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
import torch

/home/kimjaehyun/anaconda3/envs/nlp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set model name for loading from the transformers library

# # Yanolja/EEVE
# model_name = "yanolja/EEVE-Korean-Instruct-10.8B-v1.0"
# tokenizer_name = "yanolja/EEVE-Korean-Instruct-10.8B-v1.0"

#  maywell/Synatra-V0.1-7B
model_name = "maywell/Synatra-V0.1-7B"
tokenizer_name = "maywell/Synatra-V0.1-7B"

In [3]:
# Load model

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16, # half-precision to speed up inference
)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name,
                                          device_map="auto")

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.60s/it]


In [7]:
# Set text

prompt_template = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.\nHuman: {prompt}\nAssistant:\n"
text = "한국의 수도는 어디인가요? 아래 선택지 중 골라주세요.\n\n(A) 경성\n(B) 부산\n(C) 평양\n(D) 서울\n(E) 전주"

In [8]:
# Process through model and obtain model output
with torch.no_grad():
    model_inputs = tokenizer(prompt_template.format(prompt=text), return_tensors="pt").to("cuda")

    model.generation_config.pad_token_id = model.generation_config.eos_token_id  # To prevent early ending of sentence
    outputs = model.generate(**model_inputs, max_new_tokens=256)
    # outputs = model.generate(**model_inputs, pad_token_id=tokenizer.eos_token_id)
output_text = tokenizer.batch_decode(outputs, skip_special_token=True)[0]
print(output_text)

<s> A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.
Human: 한국의 수도는 어디인가요? 아래 선택지 중 골라주세요.

(A) 경성
(B) 부산
(C) 평양
(D) 서울
(E) 전주
Assistant:

AI:  한국의 수도는 (A) 경성입니다. 경성은 서울특별시를 의미하며, 이 지역은 국가 정부 기관, 대통령 및 대한민국 국회의 본부가 위치해 있습니다. 다른 옵션은 대도시나 지방 자치단체의 수도가 아닙니다.</s>


In [6]:
# Generate tokens one by one and print them
with torch.no_grad():
    model_inputs = tokenizer(prompt_template.format(prompt=text), return_tensors="pt").to("cuda")
    output_ids = model_inputs['input_ids']
    while True:
        outputs = model.generate(output_ids, max_new_tokens=1, early_stopping=True)
        new_token_id = outputs[0, -1:]
        output_ids = torch.cat([output_ids, new_token_id.unsqueeze(0)], dim=-1)
        
        # Decode and print the latest token
        new_token = tokenizer.decode(new_token_id, skip_special_tokens=True)
        print(new_token, end=' ', flush=True)
        
        # Stop if the generated token is eos_token
        if new_token_id.item() == tokenizer.eos_token_id:
            break

/home/kimjaehyun/anaconda3/envs/nlp/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



 AI :   한 국 의  수 도 는 ( A )  경 성 입 니 다 .  경 성 은  서 울 특 별 시 를  의 미 하 며 ,  이  지 역 은  국 가  정 부  기 관 ,  대 통 령  및  대 한 민 국  국 회 의  본 부 가  위 치 해  있 습 니 다 .  다 른  옵 션 은  대 도 시 나  지 방  자 치 단 체 의  수 도 가  아 � � � 니 다 .  